In [40]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set global variables
PROJECT = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks"
OUTPUTS = rf"{PROJECT}\Outputs"
DATA = rf"{PROJECT}\Data"
DATA_IN = rf"{DATA}\Data_in"
DATA_PROC = rf"{DATA}\Data_proc"
DATA_OUT = rf"{DATA}\Data_out"
ERA5_DATA = rf"Z:\WB Data\ERA5 Reanalysis\monthly"

## Variable definitions

- Drought: Monthly cumulative precipitation **2.5 standard deviations below** the monthly historical average, considering 30 years to estimate historical averages and SD
- Excessive Rain: Monthly cumulative precipitation **2.5 standard deviations above** the monthly historical average, considering 30 years to estimate historical averages and SD
- Heatwave: More than 6 days with temperature **over 40 degrees** and 2.5 standard deviations higher than monthly historical average, considering 30 years to estimate historical averages and SD. Furthermore, **warm spell** duration index is higher than 6 days.
- Coldwave: More than 6 days with temperature **below 0 degrees** and 2.5 standard deviations lower than monthly historical average, considering 30 years to estimate historical averages and SD. Furthermore, **cold spell** duration index is higher than 6 days.
  ​

In [58]:
files = os.listdir(ERA5_DATA)
datasets = []
for file in tqdm(files):
    ds = xr.open_dataset(os.path.join(ERA5_DATA, file))
    datasets += [ds]
precipitation = xr.concat(datasets, dim="time")
precipitation.to_netcdf(os.path.join(DATA_OUT, "ERA5_monthly_1970-2021.nc"))

100%|██████████| 51/51 [00:05<00:00,  9.82it/s]
C:\Users\ofici\AppData\Roaming\Python\Python311\site-packages\xarray\core\concat.py:532: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [71]:
# Longitude is in range 0-360, with 0 at Greenwich. We need to transform it to -180 to 180
def transform_longitude(longitude):
    if longitude > 180:
        return longitude - 360
    else:
        return longitude

precipitation['longitude'] = precipitation['longitude'].to_series().apply(transform_longitude).values
precipitation = precipitation.sortby('longitude').sortby("latitude")


In [84]:
precipitation = precipitation.rename({'longitude': 'lon', 'latitude': 'lat'})
precipitation

<xarray.Dataset>
Dimensions:  (lon: 1440, lat: 721, time: 612)
Coordinates:
  * lon      (lon) float64 -179.8 -179.5 -179.2 -179.0 ... 179.5 179.8 180.0
  * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * time     (time) datetime64[ns] 1970-01-01 1970-02-01 ... 2020-12-01
Data variables:
    t2m      (time, lat, lon) float32 243.6 243.6 243.6 ... 252.3 252.3 252.3
    tp       (time, lat, lon) float32 3.953e-05 3.953e-05 ... 0.0005389
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-13 13:46:22 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [90]:
precipitation["tp"]

<xarray.DataArray 'tp' (time: 612, lat: 721, lon: 1440)>
array([[[3.95253301e-05, 3.95253301e-05, 3.95253301e-05, ...,
         3.95253301e-05, 3.95253301e-05, 3.95253301e-05],
        [4.70541418e-05, 4.70541418e-05, 4.70541418e-05, ...,
         4.70541418e-05, 4.70541418e-05, 4.70541418e-05],
        [5.08204103e-05, 5.08204103e-05, 5.08204103e-05, ...,
         5.08204103e-05, 5.08204103e-05, 5.08204103e-05],
        ...,
        [4.00919467e-04, 4.00919467e-04, 4.00919467e-04, ...,
         4.02800739e-04, 4.00919467e-04, 4.00919467e-04],
        [4.00919467e-04, 4.00919467e-04, 4.00919467e-04, ...,
         4.00919467e-04, 4.00919467e-04, 4.00919467e-04],
        [4.04685736e-04, 4.04685736e-04, 4.04685736e-04, ...,
         4.04685736e-04, 4.04685736e-04, 4.04685736e-04]],

       [[4.51728702e-05, 4.51728702e-05, 4.51728702e-05, ...,
         4.51728702e-05, 4.51728702e-05, 4.51728702e-05],
        [5.83492219e-05, 5.83492219e-05, 5.83492219e-05, ...,
         5.83492219e-05, 5.83492219e-05, 5.83492219e-05],
        [5.64642251e-05, 5.64642251e-05, 5.64642251e-05, ...,
         5.64642251e-05, 5.64642251e-05, 5.64642251e-05],
...
        [1.57632679e-03, 1.57632679e-03, 1.57632679e-03, ...,
         1.57632679e-03, 1.57632679e-03, 1.57632679e-03],
        [1.67560577e-03, 1.67560577e-03, 1.67560577e-03, ...,
         1.67560577e-03, 1.67560577e-03, 1.67560577e-03],
        [1.73030794e-03, 1.73030794e-03, 1.73030794e-03, ...,
         1.73030794e-03, 1.73030794e-03, 1.73030794e-03]],

       [[3.30254436e-04, 3.30254436e-04, 3.30254436e-04, ...,
         3.30254436e-04, 3.30254436e-04, 3.30254436e-04],
        [4.05222178e-04, 4.05222178e-04, 4.05222178e-04, ...,
         4.05222178e-04, 4.05222178e-04, 4.05222178e-04],
        [4.01169062e-04, 4.01169062e-04, 4.01169062e-04, ...,
         4.01169062e-04, 4.01169062e-04, 4.01169062e-04],
        ...,
        [4.59931791e-04, 4.57905233e-04, 4.57905233e-04, ...,
         4.59931791e-04, 4.59931791e-04, 4.59931791e-04],
        [5.04501164e-04, 5.04501164e-04, 5.04501164e-04, ...,
         5.04501164e-04, 5.04501164e-04, 5.04501164e-04],
        [5.38945198e-04, 5.38945198e-04, 5.38945198e-04, ...,
         5.38945198e-04, 5.38945198e-04, 5.38945198e-04]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 -179.8 -179.5 -179.2 -179.0 ... 179.5 179.8 180.0
  * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * time     (time) datetime64[ns] 1970-01-01 1970-02-01 ... 2020-12-01
Attributes:
    units:      m
    long_name:  Total precipitation

In [92]:
import climate_indices

## User guide to SPI: https://digitalcommons.unl.edu/cgi/viewcontent.cgi?article=1208&context=droughtfacpub
#   It is recommended to use SPI-9 or SPI-12 to compute droughts.
#   "SPI values below -1.5 for these timescales (SPI-9) are usually a good indication that dryness is having a significant impact on
#    agriculture and may be affecting other sectors as well."

# Ignore negative values, they are normal: https://confluence.ecmwf.int/display/UDOC/Why+are+there+sometimes+small+negative+precipitation+accumulations+-+ecCodes+GRIB+FAQ

# Parameters
distribution = climate_indices.indices.Distribution.gamma
data_start_year = 1990
calibration_year_initial = 1990
calibration_year_final = 2020
periodicity = climate_indices.compute.Periodicity.monthly


da_precip_groupby = precipitation["tp"].stack(point=('lat', 'lon')).groupby('point')

# apply SPI to each `point`
spis = []
for i in [1, 3, 6, 12]:
    print(f"Computing SPI-{i}")
    da_spi = xr.apply_ufunc(climate_indices.indices.spi,
                            da_precip_groupby,
                            i,
                            distribution,
                            data_start_year,
                            calibration_year_initial,
                            calibration_year_final,
                            periodicity)
    da_spi = da_spi.unstack('point').rename(f"spi{i}")
    spis += [da_spi]

In [18]:
#### Construct indicators
climate_data = xr.combine_by_coords(spis + [precipitation["t2m"]])

## Export data
climate_data.to_netcdf(rf"{DATA_OUT}/Climate_shocks_v3_previous_months.nc")

In [ ]:
import pandas as pd
pd.read_csv(r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\births_climate_20000.csv")